<a href="https://colab.research.google.com/github/datecru/House-Prices-Prediction-using-TF/blob/main/House_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy
!pip install tensorflow
!pip install matplotlib
!pip install pandas
!pip install seaborn
!pip install scikit-learn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from tensorflow.keras.regularizers import l2

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test (1).csv
Saving train.csv to train (1).csv


In [ ]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df_train.head()
df_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [ ]:
df_train_numeric = df_train.select_dtypes(include=[np.number])
df_train_numeric.head()

df_test_numeric = df_test.select_dtypes(include=[np.number])
df_test_numeric.head

<bound method NDFrame.head of         Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  \
0     1461          20         80.0    11622            5            6   
1     1462          20         81.0    14267            6            6   
2     1463          60         74.0    13830            5            5   
3     1464          60         78.0     9978            6            6   
4     1465         120         43.0     5005            8            5   
...    ...         ...          ...      ...          ...          ...   
1454  2915         160         21.0     1936            4            7   
1455  2916         160         21.0     1894            4            5   
1456  2917          20        160.0    20000            5            7   
1457  2918          85         62.0    10441            5            5   
1458  2919          60         74.0     9627            7            5   

      YearBuilt  YearRemodAdd  MasVnrArea  BsmtFinSF1  ...  GarageArea  \
0          1961          1961         0.0       468.0  ...       730.0   
1          1958          1958       108.0       923.0  ...       312.0   
2          1997          1998         0.0       791.0  ...       482.0   
3          1998          1998        20.0       602.0  ...       470.0   
4          1992          1992         0.0       263.0  ...       506.0   
...         ...           ...         ...         ...  ...         ...   
1454       1970          1970         0.0         0.0  ...         0.0   
1455       1970          1970         0.0       252.0  ...       286.0   
1456       1960          1996         0.0      1224.0  ...       576.0   
1457       1992          1992         0.0       337.0  ...         0.0   
1458       1993          1994        94.0       758.0  ...       650.0   

      WoodDeckSF  OpenPorchSF  EnclosedPorch  3SsnPorch  ScreenPorch  \
0            140            0              0          0          120   
1            393           36              0          0            0   
2            212           34              0          0            0   
3            360           36              0          0            0   
4              0           82              0          0          144   
...          ...          ...            ...        ...          ...   
1454           0            0              0          0            0   
1455           0           24              0          0            0   
1456         474            0              0          0            0   
1457          80           32              0          0            0   
1458         190           48              0          0            0   

      PoolArea  MiscVal  MoSold  YrSold  
0            0        0       6    2010  
1            0    12500       6    2010  
2            0        0       3    2010  
3            0        0       6    2010  
4            0        0       1    2010  
...        ...      ...     ...     ...  
1454         0        0       6    2006  
1455         0        0       4    2006  
1456         0        0       9    2006  
1457         0      700       7    2006  
1458         0        0      11    2006  

[1459 rows x 37 columns]>

In [ ]:
df_train_numeric = df_train_numeric.fillna(df_train_numeric.mean())
df_test_numeric = df_test_numeric.fillna(df_test_numeric.mean())

In [ ]:
if 'SalePrice' in df_train_numeric.columns:
    x_train = df_train_numeric.drop('SalePrice', axis=1).values
    y_train = df_train_numeric['SalePrice'].values

else:
    x_train = df_train_numeric.values

x_test = df_test_numeric.values

#Normalize Data

In [ ]:
scaler = StandardScaler()

x_train_scaled = scaler.fit_transform(x_train)

x_test_scaled = scaler.transform(x_test)


#Create Model

In [ ]:
def create_model(input_dim):
  model = Sequential()

  model.add(Dense(128, activation='relu', input_dim=input_dim, kernel_regularizer=l2(0.01)))
  model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
  model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
  model.add(Dense(1, activation='linear'))

  optimizer = Adam(learning_rate=0.001)
  model.compile(optimizer=optimizer, loss='mean_squared_error')
  return model

model = create_model(x_train_scaled.shape[1])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 128)               4864      
                                                                 
 dense_22 (Dense)            (None, 64)                8256      
                                                                 
 dense_23 (Dense)            (None, 64)                4160      
                                                                 
 dense_24 (Dense)            (None, 1)                 65        
                                                                 
Total params: 17345 (67.75 KB)
Trainable params: 17345 (67.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = model.fit(x_train_scaled, y_train, epochs=1000, batch_size=32, validation_split=0.2)

Epoch 1/1000
37/37 [==============================] - 0s 6ms/step - loss: 682672640.0000 - val_loss: 2153801472.0000
Epoch 2/1000
37/37 [==============================] - 0s 4ms/step - loss: 681332032.0000 - val_loss: 2186402816.0000
Epoch 3/1000
37/37 [==============================] - 0s 4ms/step - loss: 681105600.0000 - val_loss: 2171532544.0000
Epoch 4/1000
37/37 [==============================] - 0s 6ms/step - loss: 678775936.0000 - val_loss: 2168969216.0000
Epoch 5/1000
37/37 [==============================] - 0s 4ms/step - loss: 688946112.0000 - val_loss: 2163174400.0000
Epoch 6/1000
37/37 [==============================] - 0s 4ms/step - loss: 674012864.0000 - val_loss: 2140413184.0000
Epoch 7/1000
37/37 [==============================] - 0s 5ms/step - loss: 669216832.0000 - val_loss: 2166130432.0000
Epoch 8/1000
37/37 [==============================] - 0s 3ms/step - loss: 669475072.0000 - val_loss: 2132543488.0000
Epoch 9/1000
37/37 [==============================] - 0s 3ms/ste

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
y_val_pred = model.predict(x_train_scaled)

y_val_pred = y_val_pred.flatten()


r2_val = r2_score(y_train, y_val_pred)
print("R2 Score:", r2_val)
print("Prediction on data validation:")
print(y_val_pred[:10])

46/46 [==============================] - 0s 3ms/step
R2 Score: 0.884580625339368
Prediction on data validation:
[210694.34 199376.61 197933.48 145318.83 278203.75 142572.58 302653.8
 198709.36 131723.92 117680.29]


In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

r2_scores_cv = []

for train_index, val_index in kf.split(x_train_scaled):
    x_train_cv, x_val_cv = x_train_scaled[train_index], x_train_scaled[val_index]
    y_train_cv, y_val_cv = y_train[train_index], y_train[val_index]

    model = create_model(input_dim=x_train_scaled.shape[1])

    model.fit(x_train_cv, y_train_cv, epochs=50, batch_size=32, validation_data=(x_val_cv, y_val_cv) )

    y_pred_val_cv = model.predict(x_val_cv).flatten()

    r2_val_cv = r2_score(y_val_cv, y_pred_val_cv)
    r2_scores_cv.append(r2_val_cv)

print("R2 Scores:", r2_scores_cv)

Epoch 1/50
37/37 [==============================] - 2s 13ms/step - loss: 38883598336.0000 - val_loss: 39646687232.0000
Epoch 2/50
37/37 [==============================] - 0s 7ms/step - loss: 38849265664.0000 - val_loss: 39555547136.0000
Epoch 3/50
37/37 [==============================] - 0s 8ms/step - loss: 38592028672.0000 - val_loss: 39021043712.0000
Epoch 4/50
37/37 [==============================] - 0s 6ms/step - loss: 37552230400.0000 - val_loss: 37190049792.0000
Epoch 5/50
37/37 [==============================] - 0s 7ms/step - loss: 34632122368.0000 - val_loss: 32692355072.0000
Epoch 6/50
37/37 [==============================] - 0s 9ms/step - loss: 28652773376.0000 - val_loss: 24951234560.0000
Epoch 7/50
37/37 [==============================] - 0s 6ms/step - loss: 20142092288.0000 - val_loss: 15665549312.0000
Epoch 8/50
37/37 [==============================] - 0s 4ms/step - loss: 12705800192.0000 - val_loss: 9233199104.0000
Epoch 9/50
37/37 [==============================] - 0s 4

In [ ]:
model = create_model(input_dim=x_train_scaled.shape[1])

model.fit(x_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2)

y_train_pred = model.predict(x_train_scaled).flatten()

r2_train = r2_score(y_train, y_train_pred)
print("R2 Score:", r2_train)

Epoch 1/100
37/37 [==============================] - 1s 8ms/step - loss: 38817480704.0000 - val_loss: 39911780352.0000
Epoch 2/100
37/37 [==============================] - 0s 3ms/step - loss: 38787510272.0000 - val_loss: 39833075712.0000
Epoch 3/100
37/37 [==============================] - 0s 3ms/step - loss: 38540394496.0000 - val_loss: 39383982080.0000
Epoch 4/100
37/37 [==============================] - 0s 4ms/step - loss: 37526597632.0000 - val_loss: 37818871808.0000
Epoch 5/100
37/37 [==============================] - 0s 3ms/step - loss: 34757799936.0000 - val_loss: 34088247296.0000
Epoch 6/100
37/37 [==============================] - 0s 3ms/step - loss: 29044975616.0000 - val_loss: 27642687488.0000
Epoch 7/100
37/37 [==============================] - 0s 4ms/step - loss: 20408092672.0000 - val_loss: 19800061952.0000
Epoch 8/100
37/37 [==============================] - 0s 4ms/step - loss: 12164556800.0000 - val_loss: 14616207360.0000
Epoch 9/100
37/37 [=============================

In [ ]:
y_pred = model.predict(x_test_scaled)

y_pred = y_pred.flatten()

print("Prediction on data test:")
print(y_pred[:10])

46/46 [==============================] - 0s 2ms/step
Prediction on data test:
[110566.43 161088.95 179330.58 201613.92 185824.28 182598.58 193031.73
 176082.61 188880.69 127238.01]


In [ ]:
cv_score = r2_score(y_val_cv, y_pred_val_cv)
print("Test Score:", cv_score)

train_score = r2_score(y_train, model.predict(x_train_scaled))
print("Train Score:", train_score)

Test Score: 0.8037703542024343
46/46 [==============================] - 0s 2ms/step
Train Score: 0.8445544439730238


In [ ]:
submission = pd.DataFrame({'Id': df_test['Id'], 'SalePrice': y_pred})

submission.to_csv('predictions.csv', index=False)


In [ ]:
df_prediction = pd.read_csv('predictions.csv')

df_prediction.head()


,Id,SalePrice
0,1461,110566.43
1,1462,161088.95
2,1463,179330.58
3,1464,201613.92
4,1465,185824.28
